In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
# default: Load the model on the available device(s)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="cuda",
)

# default processer
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


ModuleNotFoundError: No module named 'transformers'

In [1]:
import pandas as pd
from tqdm import tqdm

In [5]:
import zipfile

zip_path = 'deeplearningchallenge.zip'
extract_path = './data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("압축 해제 완료")

압축 해제 완료


In [7]:
df =pd.read_parquet('data/deep_chal_multitask_dataset.parquet')

In [8]:
df.head(10)

,input_type,task,input,output,question
0,image,captioning,https://i.pinimg.com/originals/42/97/1b/42971b...,\nThe image is a painting of a man in a purple...,
1,image,captioning,https://pulpcovers.com/wp-content/uploads/2014...,"\nThe image is the cover of a book titled ""The...",
2,image,captioning,https://pulpcovers.com/wp-content/uploads/2016...,"\nThe image is the cover of a book titled ""The...",
3,image,captioning,https://i.pinimg.com/originals/35/c4/bb/35c4bb...,\nThe image is a painting of a spaceship flyin...,
4,image,captioning,https://pulpcovers.com/wp-content/uploads/2011...,"\nThe image is the cover of a book titled ""Wes...",
5,image,captioning,https://pulpcovers.com/wp-content/uploads/2020...,\nThe image is the cover of a magazine titled ...,
6,image,captioning,https://pulpcovers.com/wp-content/uploads/2012...,"\nThe image is the cover of a book titled ""The...",
7,image,captioning,https://pulpcovers.com/wp-content/uploads/2013...,\nThe image is a painting of a man and a woman...,
8,image,captioning,https://i.pinimg.com/originals/2a/67/71/2a6771...,\nThe image is a digital illustration of a spa...,
9,image,captioning,https://pulpcovers.com/wp-content/uploads/2015...,\nThe image is the cover of a magazine titled ...,


In [4]:
 test_df = pd.read_parquet('data/deep_chal_multitask_dataset_test.parquet')

In [5]:
test_df.iloc[500]

input_type                                                image
task                                                        vqa
input         iVBORw0KGgoAAAANSUhEUgAABvsAAAjuCAAAAAABWKp2AA...
question          Which food contains 30 mg./100g. of Sodium  ?
Name: 500, dtype: object

In [6]:
batches = []
for index, row in tqdm(test_df.iterrows()):
    messages = [{
        "role": "user",
        "content": [
            {
                "type": row['input_type'],
                row['input_type']: row['input'],
            },
            {"type": "text", "text": "Do the task : " + row['task']},
            {"type": "text", "text": "use this if task is qa task \n question: " + row['question']},
        ],
    }]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    try:
        image_inputs, video_inputs = process_vision_info(messages)
    except Exception as e:
        print(f"Error processing vision info for row {index}: {str(e)}")
        messages = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": row['input'],
            },
            {"type": "text", "text": "Do the task : " + row['task']},
        ],
        }]
        image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    batches.append(inputs)

11it [00:02,  6.72it/s]/workspace/.venv/lib/python3.11/site-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (97915360 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
71it [00:21,  1.27it/s]

Error processing vision info for row 69: cannot identify image file <_io.BytesIO object at 0x7bbbd4972ca0>


90it [00:26,  2.28it/s]

Error processing vision info for row 88: cannot identify image file <_io.BytesIO object at 0x7bbb5cd23560>


268it [01:15,  1.46it/s]

...

359it [01:39,  1.90it/s]

Error processing vision info for row 357: cannot identify image file <_io.BytesIO object at 0x7bbb5cd04d10>


379it [01:46,  2.10it/s]

Error processing vision info for row 377: cannot identify image file <_io.BytesIO object at 0x7bbb5cfd6890>


530it [02:15, 34.97it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

586it [02:15, 85.79it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

634it [02:16, 94.17it/s]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
Serve

In [13]:


# short_test_df= test_df.iloc[:10]
results=[]
for index, batch in tqdm(enumerate(batches)):
    inputs=batch
    inputs = inputs.to("cuda")
    
    # Inference: Generation of the output
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    del inputs
    torch.cuda.empty_cache()
    results.append({"ID" : index , "Output":output_text})

2500it [1:49:16,  2.62s/it]


In [15]:
result =pd.DataFrame(results)

In [80]:
import gc
del batches
del model
del results

In [23]:
result

,id,output
0,0,The image appears to be a vintage illustration...
1,1,"The image is of a book cover titled ""Young Lon..."
2,2,"The image is a cover of ""Detective Fiction Wee..."
3,3,The image is a cover of a pulp fiction magazin...
4,4,The image appears to be a vintage poster adver...
...,...,...
2495,2495,"The question ""Why was Alan going to the farm t..."
2496,2496,"No, she does not believe him."
2497,2497,One instrument that uses A4 (440 Hz) to tune i...
2498,2498,The wallet was found by Prince.


In [81]:
torch.cuda.empty_cache()

In [22]:
result = result.rename(columns={"ID": "id"})          # ID → id
result= result.rename(columns={"Output":"output"})

In [25]:
result.to_csv('subimission-08021143.csv')

SyntaxError: invalid syntax (3905757731.py, line 1)